# HIPÓTESIS: En qué pais es menos arriesgado salir de pesca

##### Importo bibliotecas y el archivo csv

In [1]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv("input/attacks.csv",encoding='ISO-8859-1')
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


##### Limpio el DF corrigiendo errores para facilitar el análisis de los datos

In [2]:
print(df.shape)
df.columns

(25723, 24)


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

##### Corrijo los nombres de las columnas

In [3]:
df_col = df.rename(columns={'Sex ':'Sex','Species ':'Species','Fatal (Y/N)':'Fatal'})

######  Analizo los elementos nulos de cada columna

In [4]:
null_cols = df_col.isnull().sum()

##### Elimino las columnas que no necesito
- Elimino las columnas 'Unnamed:22' y 'Unnamed:23' porque sus valores son 'Nan'.
- Elimino 'href formula', 'href', 'pdf', 'Investigator or source' y 'original order' porque no considero que su contenido aporte información para mi análisis.
- Elimino la columna 'pdf' ya que sus valores son los nombres de los archivos pero no puedo acceder a ellos.
- Elimino 'Case Number', 'Case Numer.1' y 'Case Number.2' porque contienen la misma info que podemos encontrar en 'Date'

In [5]:
clean1 = df_col.drop(columns=['Case Number','Unnamed: 22', 'Unnamed: 23', 'Investigator or Source','pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'])
clean1.shape

(25723, 14)

In [6]:
clean1['Fatal'] = clean1['Fatal'].str.replace(' N', 'N')
#clean1 = clean1[clean1["Sex"].isin(['M', 'F'])]
clean1 = clean1[clean1.Year != 0]
clean2 = clean1
clean2.shape

(25598, 14)

In [7]:
clean2['Date'] = clean1['Date'].str.replace('-', ' ')
clean2['Date'] = clean1['Date'].str.replace('\d+', '')
clean2['Date'] = clean1['Date'].str.replace('Reported', '')
clean2['Date'] = clean1['Date'].str.replace('Early', '')
clean2['Date'] = clean1['Date'].str.replace('Late', '')
clean2['Date'] = clean1['Date'].str.replace(' ', '')
clean2['Date'] = clean1['Date'].str.replace('Summer, summer, summerof ', '')

In [12]:
clean3 = clean2[clean2.Date != ""]
clean3 = clean3[clean3["Date"].isin(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Dec'])]
clean3 = clean3.rename(columns={'Date':'Month'})

In [13]:
clean3['Fatal'].value_counts()

N          3994
Y          1173
UNKNOWN      44
M             1
2017          1
Name: Fatal, dtype: int64

In [23]:
clean4 = clean3[clean3.Fatal != "UNKNOWN"]
clean4 = clean4[clean4.Fatal != "2017"]
clean4 = clean4[clean4.Fatal != "M"]
display(clean4.head())

,Month,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species
0,Jun,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,Jun,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,Jun,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,Jun,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,Jun,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"


In [15]:
clean4['Country'].value_counts()

USA                         2103
AUSTRALIA                   1245
SOUTH AFRICA                 543
NEW ZEALAND                  115
BRAZIL                       109
                            ... 
DIEGO GARCIA                   1
NEVIS                          1
PERU                           1
SUDAN?                         1
NORTHERN MARIANA ISLANDS       1
Name: Country, Length: 184, dtype: int64

In [20]:
clean4['Activity'].value_counts()

Surfing                                     946
Swimming                                    805
Fishing                                     392
Spearfishing                                298
Bathing                                     144
                                           ... 
Free diving with a pod of dolphins            1
Casting a net                                 1
Paddle Skiing                                 1
Conch diver                                   1
Went over side of boat at trochus ground      1
Name: Activity, Length: 1360, dtype: int64

##### DF limpio
Elimino columnas que no voy a necesitar para el análisis y me quedo con un nuevo DF limpio
Para el análisis se van a comparar los tres paises con mayor número de registros de ataques (USA, AUSTRALIA y SOUTH AFRICA) y se van a eliminar columnas que no aportan información relevante como los datos relacionados con el usuario.

In [69]:
df_clean = clean4.drop(columns=['Name','Age','Time','Type','Sex','Species'])
df_clean = df_clean[(df_clean.Country=='USA') | (df_clean.Country=='AUSTRALIA') | (df_clean.Country=='SOUTH AFRICA')]

In [70]:
df_clean['Area']=df_clean['Area'].replace(np.nan,'')
df_clean['Location']=df_clean['Location'].replace(np.nan,'')
df_clean['Activity']=df_clean['Activity'].replace(np.nan,'')
df_clean['Injury']=df_clean['Injury'].replace(np.nan,'')
df_clean['Fatal']=df_clean['Fatal'].replace(np.nan,'')

In [71]:
df_clean.drop(df_clean[df_clean.Year.isnull()].index, inplace=True)

In [72]:
display(df_clean.head())

,Month,Year,Country,Area,Location,Activity,Injury,Fatal
0,Jun,2018.0,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,Jun,2018.0,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N
2,Jun,2018.0,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N
3,Jun,2018.0,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N
5,Jun,2018.0,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,"No injury, board bitten",N


##### Análisis estadístico de los años de los ataques registrados

In [76]:
df_clean["Year"].describe()

count    3890.000000
mean     1975.597429
std        40.243651
min      1751.000000
25%      1956.000000
50%      1989.000000
75%      2007.000000
max      2018.000000
Name: Year, dtype: float64

###### Para el análisis nos quedamos con los datos desde 1960, aproximadamente el percentil 25, ya que en los 200 años anteriores no hay casi registros

In [80]:
df_clean_year = df_clean.drop(df_clean[df_clean['Year']<1960].index)

In [81]:
df_clean_year["Year"].describe()

count    2818.000000
mean     1996.183818
std        17.151011
min      1960.000000
25%      1984.000000
50%      2001.000000
75%      2011.000000
max      2018.000000
Name: Year, dtype: float64

In [88]:
bins = list(range(1959,2029,10))
labels = list(range(1960,2020,10))
df_clean_year["Decade"] = pd.cut(df_clean_year.Year,bins=bins,labels=labels)

In [91]:
shark=df_clean_year[['Month','Year','Decade','Country','Area','Location','Activity','Injury','Fatal']]
display(shark.head())

,Month,Year,Decade,Country,Area,Location,Activity,Injury,Fatal
0,Jun,2018.0,2010,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,Jun,2018.0,2010,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N
2,Jun,2018.0,2010,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N
3,Jun,2018.0,2010,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N
5,Jun,2018.0,2010,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,"No injury, board bitten",N


In [93]:
shark.to_csv("output/sharks.csv")